Poniżej dokonuję regresji na współczynnikach wielomianu 6-tego stopnia. Wyniki po raz kolejny uległy poprawieniu, najprawdopodobniej dzięki zastosowaniu także współczynników dla wielomianu opisującego deltę.

In [120]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from scipy.signal import find_peaks
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

folder_path = os.path.join("datasets", "Si_jaw_delta", "")
os.makedirs(folder_path, exist_ok=True)
# Get a list of all items in the folder
all_items = os.listdir(folder_path)

# Filter the list to include only files
files = [item for item in all_items if os.path.isfile(os.path.join(folder_path, item))]


polyFeatures = PolynomialFeatures(degree=6)
polyReg = LinearRegression()
delteFeature = PolynomialFeatures(degree=6)
deltaReg = LinearRegression()
newDataFrame = pd.DataFrame()

for i in files:
    dataHelper = pd.read_csv(folder_path + i, sep='\t', header=None, index_col=False)
    info = i.split('_')
    T = info[0]
    A = info[1]
    B = info[2]
    C = info[3]
    dataHelper = dataHelper.drop(index = [0])
    dataHelper = dataHelper.drop(columns = [7])
    dataHelper.columns = ['wavelength','psi65', 'del65', 'psi70', 'del70','psi75','del75']
    dataHelper['T'] = T
    dataHelper['A'] = A
    dataHelper['B'] = B
    C = C.removesuffix(".txt")
    
    if("-" in C):
        C = C.removesuffix("e-")
        C = float(C) * 10**-5
    elif("e" in C):
        C = C.removesuffix("e")
        C = float(C) * 10**-5

    if(float(C)>1):
        C = float(C) * 10**-5
    
    dataHelper['C'] = C
    psi65 =dataHelper['psi65']
    wv = dataHelper['wavelength']
    vwPoly = polyFeatures.fit_transform(wv.values.reshape(-1,1))
    del65 = dataHelper['del65']
    polyReg.fit(vwPoly, psi65)
    deltaReg.fit(vwPoly, del65)
    a1 = polyReg.coef_[1]
    a2 = polyReg.coef_[2]
    a3 = polyReg.coef_[3]
    a4 = polyReg.coef_[4]
    a5 = polyReg.coef_[5]
    a6 = polyReg.coef_[6]
    d1 = deltaReg.coef_[1]
    d2 = deltaReg.coef_[2]
    d3 = deltaReg.coef_[3]
    d4 = deltaReg.coef_[4]
    d5 = deltaReg.coef_[5]
    d6 = deltaReg.coef_[6]
    peaksPsi = peaks, _ = find_peaks(dataHelper['psi65'])
    peaksDel = peaks, _ = find_peaks(dataHelper['del65'])
    data = pd.Series([a1,a2,a3,a4,a5,a6,d1,d2,d3,d4,d5,d6, polyReg.intercept_,deltaReg.intercept_,len(peaksPsi),len(peaksDel),T,A,B,C])
    df = data.to_frame().T
    newDataFrame = pd.concat([newDataFrame, df], ignore_index=True)


newDataFrame.columns = ['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel', 'T', 'A', 'B', 'C']

newDataFrame.head()

,a1,a2,a3,a4,a5,a6,d1,d2,d3,d4,d5,d6,intercept,delIntercept,peaksPsi,peaksDel,T,A,B,C
0,-0.0,-0.0,-0.000001,0.0,-0.0,0.0,0.0,0.0,0.000003,-0.0,0.0,-0.0,44.542159,125.241061,2,2,1.199,1.4956,0.0088,0.000019
1,-0.0,-0.0,-0.000001,0.0,-0.0,0.0,0.0,0.0,0.000003,-0.0,0.0,-0.0,44.495271,124.015754,2,2,1.3980,1.4002,0.0102,0.000011
2,-0.0,-0.0,-0.000001,0.0,-0.0,0.0,0.0,0.0,0.000003,-0.0,0.0,-0.0,44.381327,122.873295,2,2,1.597,1.4454,0.0108,0.000014
3,-0.0,-0.0,-0.000001,0.0,-0.0,0.0,0.0,0.0,0.000003,-0.0,0.0,-0.0,44.215924,122.800447,2,2,1.597,1.5358,0.0116,0.000015
4,-0.0,-0.0,-0.000001,0.0,-0.0,0.0,0.0,0.0,0.000003,-0.0,0.0,-0.0,44.462451,121.794825,2,2,1.796,1.3618,0.0105,0.000015


In [121]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import joblib

coefficients = newDataFrame[['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel']]
coeffScaler = MinMaxScaler()
coefficients = coeffScaler.fit_transform(coefficients)
tScaler = MinMaxScaler()
newYT = newDataFrame['T']
YTscaled = tScaler.fit_transform(newYT.values.reshape(-1,1))

newYA = newDataFrame['A']
YAscaled = MinMaxScaler().fit_transform(newYA.values.reshape(-1,1))

newYB = newDataFrame['B']
YBscaled = MinMaxScaler().fit_transform(newYB.values.reshape(-1,1))


newYC = newDataFrame['C']
YCscaled = MinMaxScaler().fit_transform(newYC.values.reshape(-1,1))


newXT_train, newXT_test, newYT_train, newYT_test = train_test_split(coefficients, YTscaled, test_size=0.2, random_state=0)
newXA_train, newXA_test, newYA_train, newYA_test = train_test_split(coefficients, YAscaled, test_size=0.2, random_state=0)
newXB_train, newXB_test, newYB_train, newYB_test = train_test_split(coefficients, YBscaled, test_size=0.2, random_state=0)
newXC_train, newXC_test, newYC_train, newYC_test = train_test_split(coefficients, YCscaled, test_size=0.2, random_state=0)


regT = LinearRegression()
regA = LinearRegression()
regB = LinearRegression()
regC = LinearRegression()

regT.fit(newXT_train, newYT_train)
regA.fit(newXA_train, newYA_train)
regB.fit(newXB_train, newYB_train)
regC.fit(newXC_train, newYC_train)
print("RMSE dla T: ", mean_squared_error(newYT_test, regT.predict(newXT_test)))
print("wariancja dla T: ", newYT_test.var())
print("RMSE dla A: ", mean_squared_error(newYA_test, regA.predict(newXA_test)))
print("wariancja dla A: ", newYA_test.var())
print("RMSE dla B: ", mean_squared_error(newYB_test, regB.predict(newXB_test)))
print("wariancja dla B: ", newYB_test.var())
print("RMSE dla C: ", mean_squared_error(newYC_test, regC.predict(newXC_test)))
print("wariancja dla C: ", newYC_test.var())
joblib.dump(tScaler, 'Yscaler.pkl')
joblib.dump(coeffScaler, 'coeffScaler.pkl')
# Save the model
joblib.dump(regT, 'model.pkl')

RMSE dla T:  0.00042695369517264345
wariancja dla T:  0.09089467970029197
RMSE dla A:  0.032582743726836644
wariancja dla A:  0.08760718577119717
RMSE dla B:  0.06018226563181429
wariancja dla B:  0.09115435568704795
RMSE dla C:  0.08879393696081304
wariancja dla C:  0.08469877964989568


['model.pkl']

In [122]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import numpy as np
from scipy.signal import find_peaks
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
folder_path = os.path.join("datasets", "si02", "")
os.makedirs(folder_path, exist_ok=True)
# Get a list of all items in the folder
all_items = os.listdir(folder_path)

# Filter the list to include only files
files = [item for item in all_items if item.endswith('.csv')]

polyFeatures = PolynomialFeatures(degree=6)
polyReg = LinearRegression()
delteFeature = PolynomialFeatures(degree=6)
deltaReg = LinearRegression()
newDataFrame = pd.DataFrame()
for i in files:
    dataHelper = pd.read_csv(folder_path + i, sep=',')
    dataHelper.columns = ['wavelength', 'psi60', 'psi65', 'psi70','thickness','delta60','delta65','delta70']
    T = dataHelper['thickness'].values[1]
    psi65 =dataHelper['psi65']
    wv = dataHelper['wavelength']
    vwPoly = polyFeatures.fit_transform(wv.values.reshape(-1,1))
    del65 = dataHelper['delta65']
    polyReg.fit(vwPoly, psi65)
    deltaReg.fit(vwPoly, del65)
    a1 = polyReg.coef_[1]
    a2 = polyReg.coef_[2]
    a3 = polyReg.coef_[3]
    a4 = polyReg.coef_[4]
    a5 = polyReg.coef_[5]
    a6 = polyReg.coef_[6]
    d1 = deltaReg.coef_[1]
    d2 = deltaReg.coef_[2]
    d3 = deltaReg.coef_[3]
    d4 = deltaReg.coef_[4]
    d5 = deltaReg.coef_[5]
    d6 = deltaReg.coef_[6]
    peaksPsi = peaks, _ = find_peaks(dataHelper['psi65'])
    peaksDel = peaks, _ = find_peaks(dataHelper['delta65'])
    data = pd.Series([a1,a2,a3,a4,a5,a6,d1,d2,d3,d4,d5,d6, polyReg.intercept_,deltaReg.intercept_,len(peaksPsi),len(peaksDel),T])
    df = data.to_frame().T
    newDataFrame = pd.concat([newDataFrame, df], ignore_index=True)

newDataFrame.columns = ['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel', 'T']
newDataFrame.head()
print(newDataFrame)

             a1            a2        a3            a4            a5  \
0 -1.022153e-11 -4.293742e-09 -0.000001  3.039658e-09 -3.020801e-12   
1 -1.018628e-11 -4.278938e-09 -0.000001  3.021210e-09 -2.996578e-12   
2 -1.364278e-11 -7.153055e-09 -0.000002  4.701720e-09 -4.757241e-12   
3 -9.996893e-12 -4.199381e-09 -0.000001  2.988517e-09 -2.980815e-12   
4 -9.854277e-12 -4.139473e-09 -0.000001  2.942548e-09 -2.931673e-12   
5 -9.887474e-12 -4.153418e-09 -0.000001  2.829269e-09 -2.735057e-12   
6 -9.905790e-12 -4.161112e-09 -0.000001  2.958517e-09 -2.948780e-12   
7 -1.005282e-11 -4.222875e-09 -0.000001  3.001613e-09 -2.990786e-12   
8 -1.002856e-11 -4.212684e-09 -0.000001  2.993208e-09 -2.982256e-12   
9 -1.008324e-11 -4.235653e-09 -0.000001  3.001819e-09 -2.985328e-12   

             a6            d1            d2            d3            d4  \
0  1.037991e-15  2.353206e-12  9.885082e-10  2.502564e-07 -6.020007e-10   
1  1.028098e-15  2.065164e-12  8.675109e-10  2.196240e-07 -5.013437e

In [123]:
Yscaler = joblib.load('Yscaler.pkl')
Xscaler = joblib.load('coeffScaler.pkl')
model = joblib.load('model.pkl')

coefficients2 = newDataFrame[['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel']]
coefficients = Xscaler.transform(coefficients2)

newYT = newDataFrame['T']
YTscaled = Yscaler.transform(newYT.values.reshape(-1,1))

print("RMSE dla T: ", mean_squared_error(YTscaled, model.predict(coefficients)))
print("wariancja dla T: ", YTscaled.var())

RMSE dla T:  0.20870627968697536
wariancja dla T:  0.002847598570938158


In [110]:
import pandas as pd

# Pick the first row (index 0)
row = newDataFrame.iloc[0]
print(row)

a1             -1.022153e-11
a2             -4.293742e-09
a3             -1.087028e-06
a4              3.039658e-09
a5             -3.020801e-12
a6              1.037991e-15
d1              2.353206e-12
d2              9.885082e-10
d3              2.502564e-07
d4             -6.020007e-10
d5              5.539271e-13
d6             -1.833285e-16
intercept       3.925880e+01
delIntercept    1.372810e+02
peaksPsi        2.000000e+00
peaksDel        2.000000e+00
T               1.140000e+01
Name: 0, dtype: float64


In [127]:
rowX = row[['a1','a2','a3','a4','a5','a6','d1','d2','d3','d4','d5','d6','intercept','delIntercept','peaksPsi','peaksDel']]
rowY = row[['T']]
rowX = coeffScaler.transform(rowX.values.reshape(-1,1))
rowY = Yscaler.transform(rowY.values.reshape(-1,1))
regT.predict(rowX)
print("Wynik dla T: ", model.predict(rowX))
print("Wartość dla T: ", rowY)
print(model.intercept_)

C:\Users\52 Blue\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: X has 1 features, but MinMaxScaler is expecting 16 features as input.